<a href="https://colab.research.google.com/github/yukinaga/langchain/blob/main/section_3/02_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents
Agentsは独立したタスクを自動的に実行するエージェントです。  
今回のコードは、以下の公式ドキュメントを参考にしています。  
https://python.langchain.com/docs/


## ライブラリのインストール
langchain-coreとlangchain-openai、そしてwikipediaをインストールします。  

In [ ]:
!pip install langchain langchain-openai wikipedia

## API Keyの設定
OpneAIのAPIを使用するために必要な「API key」を設定します。  
以下のコードの、  
`os.environ["OPENAI_API_KEY"] = "Your API key"`  
における  
`Your API key`の箇所を、自分のAPI keyに置き換えます。  
OpenAIのAPI keyは、OpenAIのサイトで取得できます。   
https://platform.openai.com/account/api-keys


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "Your API key"

API keyの流出にはリスクがあります。  
他者に知られないように、慎重に扱ってください。

## Agentの設定
Agentを設定します。  
まずは使用するモデルを指定します。  

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

次に、プロンプトの設定を行います。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは優秀なアシスタントです。"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

次に、使用するツールを設定します。  
今回は、数学とWikipediaのツールを設定します。  



In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    tool_names=["llm-math", "wikipedia"],
    llm=chat
)

モデル、プロンプト、ツールの設定を使用し、エージェントを作成します。

In [ ]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(chat, tools, prompt)

## エージェントの利用
エージェントを利用し、様々な問題に対処します。  
以下のコードでは「AgentExecutor」を使用し、エージェントの実行環境を作成しています。

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

以下の質問をすると、Wikipediaのデータをもとに返答が作成されます。

In [ ]:
agent_executor.invoke({"input": "関ヶ原の戦いについて詳しく教えてください。"})

以下の質問には、数学のツールが自動で選択されて返答が作成されます。

In [ ]:
agent_executor.invoke({"input": "円周率を二乗した値は？"})

コードの実行後は、OpenAIのサイトでAPIの使用量を確認しましょう。  
https://platform.openai.com/account/usage